- A simple prototype
- Efficientnet-b0
- 10 epochs/32 batch_size
- ReduceLROnPlateau
- RandomCrop and Flip
- Smooth label

# 1. Import Packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc, cv2, os, warnings, random, time, math, json
from tqdm import tqdm
from sklearn.metrics import *
from sklearn.model_selection import StratifiedKFold

warnings.filterwarnings("ignore")
input_path = "/kaggle/input/"
output_path = "/kaggle/working/"

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as K
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.optimizers.schedules import *
from tensorflow.keras.activations import *
from tensorflow.keras.losses import *
from tensorflow.keras.metrics import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.regularizers import *
from tensorflow.keras.initializers import *
from tensorflow.keras.utils import get_custom_objects

In [ ]:
!pip install efficientnet
!pip install image-classifiers==1.0.0b1

from tensorflow.keras.applications import *
from efficientnet.tfkeras import *
from classification_models.tfkeras import Classifiers

In [ ]:
from tensorflow.keras.mixed_precision import experimental

experimental.set_policy(experimental.Policy("mixed_float16"))

In [ ]:
SEED = 2020
os.environ["TF_DETERMINISTIC_OPS"] = "1"
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# 2. Preprocess Dataset

In [ ]:
def preprocess(target_size=(224, 224), augment=False):
    size = (int(target_size[0] * 8 / 7), int(target_size[1] * 8 / 7))
    
    def _preprocess(filename, label):
        image = tf.io.read_file(filename)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, size, method=tf.image.ResizeMethod.BICUBIC)
        if augment:
            image = tf.image.random_crop(image, (*target_size, 3))
            image = tf.image.random_flip_left_right(image)
            image = tf.image.random_flip_up_down(image)
        else:
            image = tf.image.central_crop(image, target_size[0] / size[0])
        image = tf.clip_by_value(image, 0., 255.)
        image = image / 255.
        return image, label

    return _preprocess


class CustomDataset(object):
    def __init__(self, folds=5, fold=0, target_size=(224, 224), batch_size=32):
        self.target_size = target_size
        self.batch_size = batch_size
        
        self.autotune = tf.data.experimental.AUTOTUNE
        self.table = pd.read_csv(input_path + "cassava-leaf-disease-classification/train.csv").values
        self.classes = json.load(open(input_path + "cassava-leaf-disease-classification/label_num_to_disease_map.json", 'r'))
        
        filenames, labels = [], []
        for i, j in self.table:
            filenames.append(input_path + "cassava-leaf-disease-classification/train_images/%s" % i)
            labels.append(j)
        filenames, labels = np.asarray(filenames), np.asarray(labels)

        split = StratifiedKFold(n_splits=folds, shuffle=True, random_state=SEED).split(filenames, labels)
        labels = np.eye(len(self.classes))[labels].astype(np.float32)
        for i in range(fold): next(split)
        train_indices, val_indices = next(split)
        self.train_len, self.val_len = len(train_indices), len(val_indices)
        self.train_filenames, self.train_labels = filenames[train_indices], labels[train_indices]
        self.val_filenames, self.val_labels = filenames[val_indices], labels[val_indices]
        del filenames, labels, split, train_indices, val_indices 
        gc.collect()
        
        print("Number of train:", self.train_len, "\nNumber of val:", self.val_len)

    def __len__(self):
        return self.train_len + self.val_len
    
    def getTrainDataset(self):
        return (tf.data.Dataset.from_tensor_slices((self.train_filenames, self.train_labels))
                .shuffle(buffer_size=self.train_len, seed=SEED)
                .cache()
                .map(preprocess(target_size=self.target_size, augment=True), num_parallel_calls=self.autotune)
                .batch(self.batch_size)
                .prefetch(buffer_size=self.autotune)), self.train_len
    
    def getValidDataset(self):
        return (tf.data.Dataset.from_tensor_slices((self.val_filenames, self.val_labels))
                .map(preprocess(target_size=self.target_size), num_parallel_calls=self.autotune)
                .batch(self.batch_size)
                .cache()
                .prefetch(buffer_size=self.autotune)), self.val_len
    
    def getClasses(self):
        return self.classes

# 3. Define Model

In [ ]:
class CustomClassifier(object):
    def __init__(self, dataset):
        self.train_dataset, self.train_len = dataset.getTrainDataset()
        self.val_dataset, self.val_len = dataset.getValidDataset()
        self.classes = list(dataset.getClasses().values())

    def build(self, input_shape=(128, 128, 3)):
        self.target_size = input_shape[:-1]
        pretrained = EfficientNetB0(weights="imagenet", include_top=False)
        for layer in pretrained.layers: layer.trainable = True

        i = Input(shape=input_shape)
        x = pretrained(i)
        x = GlobalAveragePooling2D()(x)
        x = Dense(len(self.classes), use_bias=True)(x)
        o = Activation("softmax", dtype="float32")(x)

        self.clf = Model(i, o)
        self.clf.compile(
            optimizer=Adam(1e-4),
            loss=lambda x, y: categorical_crossentropy(x, y, label_smoothing=0.1),
            metrics=["accuracy"]
        )
        self.clf.summary()
        
    def fit(self, epochs=100, batch_size=32):
        self.clf.fit(
            self.train_dataset, validation_data=self.val_dataset,
            epochs=epochs, verbose=1,
            callbacks=[
                CSVLogger(output_path + "history.csv", separator=',', append=False),
                ModelCheckpoint(output_path + "model_check.h5", save_best_only=True, save_weights_only=True, monitor="val_loss"),
            ],
        )

    def predict(self):
        def metricsPrint(yTrue, yPred, classes):
            print(classification_report(yTrue, yPred, target_names=classes, digits=4))

        def cmDraw(yTrue, yPred, classes):
            cm = confusion_matrix(yTrue, yPred, labels=range(len(classes)))
            df = pd.DataFrame(cm, index=classes, columns=classes)
            plt.figure(figsize=(len(classes) + 3, len(classes) + 3))
            sns.heatmap(df, annot=True, fmt="d", cmap=plt.cm.Blues)
            plt.show()
        
        preds, targets = [], []
        for i in self.val_dataset:
            preds += list(np.argmax(self.clf.predict(i[0]), axis=-1))
            targets += list(np.argmax(i[1].numpy(), axis=-1))
        preds, targets = np.asarray(preds, dtype=np.int), np.asarray(targets, dtype=np.int)
                            
        metricsPrint(targets, preds, self.classes)
        cmDraw(targets, preds, self.classes)

# 4. Fit

In [ ]:
dataset = CustomDataset(folds=5, fold=0, target_size=(224, 224), batch_size=32)

In [ ]:
classifier = CustomClassifier(dataset)

In [ ]:
classifier.build(input_shape=(224, 224, 3))

In [ ]:
classifier.fit(epochs=10, batch_size=32)

In [ ]:
classifier.predict()